In [28]:
import pandas as pd
import numpy as np
import os
import glob
import json
import datetime
from dateutil.parser import parse
import gc
from tqdm import tqdm
tqdm.pandas()

In [2]:
#s6_df = pd.read_json("ver_releases.json")
all_files_s6 = glob.glob(os.path.join("bulk-s6" , "*.json"))
s6_df_from_each_file = (pd.read_json(f) for f in all_files_s6)
s6_df = pd.concat(s6_df_from_each_file, ignore_index=True)
del s6_df_from_each_file

In [3]:
s6_df.columns

Index(['_id', 'id', 'initiationType', 'ocid', 'date', 'tag', 'parties',
       'planning', 'tender', 'awards', 'contracts', 'language', 'buyer',
       'cycle', 'publisher', 'dependencia', 'metadata', 'metadatos',
       'publicar', 'implementation'],
      dtype='object')

In [4]:
s6_df['_id.$oid'] = s6_df['_id'].apply(lambda x: x.get('$oid'))

In [5]:
keep_cols = ['_id.$oid', "ocid", "id", "parties", "awards"]
s6_df = s6_df[keep_cols]

In [6]:
s6_df.columns

Index(['_id.$oid', 'ocid', 'id', 'parties', 'awards'], dtype='object')

In [7]:
s6_df.head()

,_id.$oid,ocid,id,parties,awards
0,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,"[{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RF...","[{'id': 9, 'title': 'SESEA/DGA/C001', 'descrip..."
1,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,"[{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RF...","[{'id': 13, 'title': 'Póliza C00021', 'descrip..."
2,62a913e7db2a665c4fce6225,ocds-ty10ed-06-06-2022-08:06:10,13,"[{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RF...","[{'id': 12, 'title': 'SESEA/DGA/C004', 'descri..."
3,62a913e7db2a665c4fce6227,ocds-ty10ed-13-06-2022-13:15:25,15,"[{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RF...","[{'id': 16, 'title': 'Servicio de Auditoria y ..."
4,62a913e7db2a665c4fce6222,ocds-ty10ed-02-06-2022-12:17:13,10,"[{'name': 'Nissan Torres Corzo', 'id': 'MX-RFC...","[{'id': 8, 'title': 'SESEA/DGA/C003', 'descrip..."


In [24]:
def parse_date(x):
    try:
        return parse(x)
    except:
        return None

def find_start_date(awards):
    start_dates = []
    end_dates = []
    if awards is not list:
        return np.datetime64('NaT'), np.datetime64('NaT')
    for award in awards:
        start_date = parse_date(award.get("contractPeriod", {}).get("startDate"))
        if start_date:
            start_dates.append(start_date)
        end_date = parse_date(award.get("contractPeriod",{}).get("endDate"))
        if end_date:
            end_dates.append(end_date)
    
    if len(start_dates) > 0:
        earliest_start = min(start_dates)
    else:
        earliest_start = np.datetime64('NaT')
        
    if len(start_dates) > 0:
        oldest_end = max(end_dates)
    else:
        oldest_end = np.datetime64('NaT')

    return earliest_start, oldest_end

def find_dates_no_processing(awards):
    start_dates = []
    end_dates = []
    if type(awards) is list:
        for award in awards:
            if type(award) is dict:
                start_date = award.get("contractPeriod", {}).get("startDate")
                start_dates.append(start_date)
                end_date = award.get("contractPeriod",{}).get("endDate")
                end_dates.append(end_date)
        return start_dates, end_dates
    else:
        return [], []
    

In [29]:
res = s6_df.awards.progress_apply(find_dates_no_processing)

100%|██████████| 1061191/1061191 [01:12<00:00, 14713.63it/s]


In [32]:
s6_df["contractPeriod_startDate"], s6_df["contractPeriod_endDate"] = zip(*res)

In [35]:
s6_df = s6_df.drop(columns=["awards"])

In [36]:
s6_df = s6_df.explode("parties")

In [37]:
s6_df.head()

,_id.$oid,ocid,id,parties,contractPeriod_startDate,contractPeriod_endDate
0,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,"{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RFC...",[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z]
0,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,"{'name': 'Nissan Torres Corzo', 'id': 'MX-RFC-...",[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z]
1,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,"{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RFC...",[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z]
1,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,"{'name': 'Grant Limpieza y mantenimiento', 'id...",[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z]
2,62a913e7db2a665c4fce6225,ocds-ty10ed-06-06-2022-08:06:10,13,"{'name': 'SESEA AGUASCALIENTES', 'id': 'MX-RFC...",[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z]


In [43]:
def extract_parties_names(parties):
    if type(parties) is dict:
        name = parties.get("name")
        contact = parties.get("contactPoint", {}).get("name")
        return name, contact
    else:
        return "", ""

In [ ]:
res_contact = s6_df.parties.progress_apply(extract_parties_names)

In [ ]:
s6_df["parties_name"], s6_df["parties_contactPoint_name"] = zip(*res_contact)

In [ ]:
s6_df = s6_df.drop(columns=["parties"])

In [40]:
s6_df = s6_df.reset_index(drop = True)

In [41]:
s6_df.head()

,_id.$oid,ocid,id,contractPeriod_startDate,contractPeriod_endDate,parties_name,parties_contactPoint_name
0,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z],SESEA AGUASCALIENTES,Monica Lizbeth de la Cruz Ortega
1,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z],Nissan Torres Corzo,Grupo Torres Corzo Automotriz de Aguascaliente...
2,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z],SESEA AGUASCALIENTES,Monica Lizbeth de la Cruz Ortega
3,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z],Grant Limpieza y mantenimiento,Grant Limpieza y mantenimiento
4,62a913e7db2a665c4fce6225,ocds-ty10ed-06-06-2022-08:06:10,13,[2022-06-14T11:21:02Z],[2022-06-14T11:21:02Z],SESEA AGUASCALIENTES,Monica Lizbeth de la Cruz Ortega


In [42]:
try:
    s6_df.to_parquet("s6_parquet.parquet")
except Exception as e:
    print(e)
    s6_df.to_hdf("s6_hdf.h5", key = "s6_df")

("Could not convert 'SHCP-1138261-2021-04-05' with type str: tried to convert to int64", 'Conversion failed for column id with type object')


C:\Users\garci\AppData\Local\Temp\ipykernel_5536\2114742651.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['_id.$oid', 'ocid', 'id', 'contractPeriod_startDate',
       'contractPeriod_endDate', 'parties_name', 'parties_contactPoint_name'],
      dtype='object')]

  s6_df.to_hdf("s6_hdf.h5", key = "s6_df")


In [22]:
#s6_df['contractPeriod_startDate'] = s6_df['contractPeriod_startDate'].apply(lambda x: pd.to_datetime(x, utc=True))
#s6_df['contractPeriod_endDate'] = s6_df['contractPeriod_endDate'].apply(lambda x: pd.to_datetime(x, utc=True))

#s6_df['contractPeriod_startDate'] = s6_df['contractPeriod_startDate'].dt.date
#s6_df['contractPeriod_endDate'] = s6_df['contractPeriod_endDate'].dt.date

In [24]:
#try:
#    s6_df.to_parquet("s6_parquet_dates.parquet")
#except:
#    s6_df.to_hdf("s6_hdf_dates.h5", key = "s6_df_dates")